<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [45]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [95]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [47]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [48]:
# use requests.get() method with the provided static_url
# assign the response to a object
r = requests.get(static_url)

Create a `BeautifulSoup` object from the HTML `response`


In [69]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(r.content, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [70]:
# Use soup.title attribute
title = soup.title.string
print(f"Page Title:{title}")

Page Title:List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [30]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [31]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
# print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [ ]:
first_launch_table

In [54]:
def extract_column_from_header(th):
    if th.string:
        return th.string.strip()
    return None

column_names = []

th_elements = first_launch_table.find_all('th')

for th in th_elements:
    name = extract_column_from_header(th)
    
    if name is not None and len(name)>0:
        column_names.append(name)

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [57]:
print(column_names)

['Flight No.', 'Launch site', 'Payload mass', 'Orbit', 'Customer', '1', '2', '3', '4', '5', '6', '7']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [97]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
#del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [87]:
print(column_names)

['Flight No.', 'Launch site', 'Payload mass', 'Orbit', 'Customer', '1', '2', '3', '4', '5', '6', '7']


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [98]:
from bs4 import BeautifulSoup
import pandas as pd

# Helper functions
def extract_date_time(cell):
    """Extract date and time from a table cell."""
    if cell:
        datatimelist = cell.text.strip().split()
        date = datatimelist[0].strip(',') if len(datatimelist) > 0 else None
        time = datatimelist[1] if len(datatimelist) > 1 else None
        return date, time
    return None, None

def extract_booster_version(cell):
    """Extract booster version from a table cell."""
    if cell:
        return cell.text.strip()
    return None

def extract_launch_site(cell):
    """Extract launch site from a table cell."""
    if cell and cell.a:
        return cell.a.text.strip()
    return None

def extract_payload(cell):
    """Extract payload from a table cell."""
    if cell and cell.a:
        return cell.a.text.strip()
    return None

def extract_payload_mass(cell):
    """Extract payload mass from a table cell."""
    if cell:
        return cell.text.strip()
    return None

def extract_orbit(cell):
    """Extract orbit from a table cell."""
    if cell and cell.a:
        return cell.a.text.strip()
    return None

def extract_customer(cell):
    """Extract customer from a table cell."""
    if cell and cell.a:
        return cell.a.text.strip()
    return None

def extract_launch_outcome(cell):
    """Extract launch outcome from a table cell."""
    if cell:
        return cell.text.strip()
    return None

def extract_booster_landing(cell):
    """Extract booster landing status from a table cell."""
    if cell:
        return cell.text.strip()
    return None

# Initialize variables
extracted_row = 0
launch_dict = {}

# Extract each table
for table_number, table in enumerate(soup.find_all('table', class_='wikitable plainrowheaders collapsible')):
    # Get table rows
    for rows in table.find_all("tr"):
        # Check if the first table heading is a number corresponding to a launch number
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False

        # Get table elements
        row = rows.find_all('td')

        # If it is a number, save cells in a dictionary
        if flag and len(row) >= 9:  # Ensure there are enough columns
            extracted_row += 1

            # Flight Number value
            launch_dict[flight_number] = {}
            launch_dict[flight_number]['Flight No.'] = flight_number

            # Extract date and time
            date, time = extract_date_time(row[0])
            launch_dict[flight_number]['Date'] = date
            launch_dict[flight_number]['Time'] = time

            # Booster version
            bv = extract_booster_version(row[1])
            launch_dict[flight_number]['Version Booster'] = bv

            # Launch Site
            launch_site = extract_launch_site(row[2])
            launch_dict[flight_number]['Launch Site'] = launch_site

            # Payload
            payload = extract_payload(row[3])
            launch_dict[flight_number]['Payload'] = payload

            # Payload Mass
            payload_mass = extract_payload_mass(row[4])
            launch_dict[flight_number]['Payload mass'] = payload_mass

            # Orbit
            orbit = extract_orbit(row[5])
            launch_dict[flight_number]['Orbit'] = orbit

            # Customer
            customer = extract_customer(row[6])
            launch_dict[flight_number]['Customer'] = customer

            # Launch outcome
            launch_outcome = extract_launch_outcome(row[7])
            launch_dict[flight_number]['Launch outcome'] = launch_outcome

            # Booster landing
            booster_landing = extract_booster_landing(row[8])
            launch_dict[flight_number]['Booster landing'] = booster_landing

# Convert dictionary to DataFrame
df = pd.DataFrame({key: pd.Series(value, dtype='object') for key, value in launch_dict.items()})

In [111]:
df_transposed = df.T

# Print the transposed DataFrame
print(df_transposed.head(20))

   Flight No. Date       Time       Version Booster     Launch Site  \
1           1    4       June  F9 v1.0[7]B0003.1[8]           CCAFS   
2           2    8   December  F9 v1.0[7]B0004.1[8]           CCAFS   
3           3   22        May  F9 v1.0[7]B0005.1[8]           CCAFS   
4           4    8    October  F9 v1.0[7]B0006.1[8]           CCAFS   
5           5    1      March  F9 v1.0[7]B0007.1[8]           CCAFS   
6           6   29  September    F9 v1.1[7]B1003[8]            VAFB   
7           7    3   December          F9 v1.1B1004           CCAFS   
8           8    6    January               F9 v1.1           CCAFS   
9           9   18      April               F9 v1.1  Cape Canaveral   
10         10   14       July               F9 v1.1  Cape Canaveral   
11         11    5     August               F9 v1.1  Cape Canaveral   
12         12    7  September       F9 v1.1B1011[8]  Cape Canaveral   
13         13   21  September       F9 v1.1B1010[8]  Cape Canaveral   
14    

In [113]:
rocket_column = 'Version Booster'  # Column name for booster version
df_falcon9 = df_transposed[df_transposed[rocket_column].str.contains('F9 v1.2', na=False)]
num_falcon9_launches = len(df_falcon9)

# Print the filtered DataFrame and count
print(f"\nNumber of Falcon 9 launches: {num_falcon9_launches}")


Number of Falcon 9 launches: 0


In [110]:
# Assuming df is your original DataFrame
df_transposed = df.T  # Transpose the DataFrame

# Check if 'landingPad' exists in the transposed DataFrame
if 'Booster landing' in df_transposed.columns:
    # Count missing values in the 'landingPad' column
    missing_landingPad = df_transposed['Booster landing'].isna().sum()
    print(f"Number of missing values in 'landingPad': {missing_landingPad}")
else:
    print("Column 'landingPad' does not exist in the transposed DataFrame.")

Number of missing values in 'landingPad': 0


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
